# 타이타닉 : 누가 살아남았을까?

## 1. 데이터 불러오기

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

from lightgbm import LGBMClassifier

In [5]:
path = "/content/drive/MyDrive/Dacon/튜토리얼/data/titanic_data"
os.chdir(path)

In [6]:
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
submission = pd.read_csv("sample_submission.csv")

In [7]:
train.shape, test.shape

((891, 11), (418, 10))

In [11]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


- 특성 설명

num---------------

PassengerID : 탑승객의 고유 아이디

Survived : 생존유무(0: 사망, 1: 생존)

Age : 나이

Pclass : 등실의 등급

SibSP : 함께 탑승한 형제자매, 아내, 남편의 수

Parch : 함께 탑승한 부모, 자식의 수

Fare : 요금

cat---------------

Name : 이름

Sex : 성별

Ticket : 티켓 번호

Cabin : 객실번호

Embarked : 배에 탑승한 위치(C = Cherbourg, Q = Queenstown, S = Southampton)

## 2. 데이터 전처리

In [12]:
corr_matrix = train.corr()
corr_matrix['Survived'].sort_values(ascending=False)

Survived    1.000000
Fare        0.257307
Parch       0.081629
SibSp      -0.035322
Age        -0.077221
Pclass     -0.338481
Name: Survived, dtype: float64

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [15]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [16]:
# 훈련데이터와 타깃 분리
target = ['Survived']
y_train = train[target]
X_train = train.drop(target, axis=1)

In [17]:
# 숫자형, 범주형 특성 나누기
cat_features = [i for i in X_train.columns if X_train[i].dtype=='object']
num_features  = [i for i in X_train.select_dtypes(exclude='object')]

(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
 ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'])

In [20]:
print(f"범주형 특성: {cat_features}")
print(f"숫자형 특성: {num_features}")

범주형 특성: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
숫자형 특성: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [18]:
train['Cabin'].nunique()

147

In [19]:
train['Ticket'].nunique()

681

In [21]:
# 불필요하다고 판단한 특성 제거
cat_features.remove('Name')
cat_features.remove('Ticket')
cat_features.remove('Cabin')

In [22]:
# nan인 특성 찾아보기
X_train.isna().sum()

Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [23]:
test.isna().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

## 3. 모델 생성 및 훈련

In [42]:
from lightgbm.sklearn import LGBMClassifier
# nan값 채워넣기 및 표준화, one-hot 인코딩
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('standard', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

model = LGBMClassifier()

# grid search로 최적의 모델 파라미터 찾기
reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('grid', GridSearchCV(model,
                                            param_grid={
                                                  'num_leaves': [7, 14, 21, 28, 31, 50],
                                                  'learning_rate': [0.1, 0.03, 0.003],
                                                  'max_depth': [-1, 3, 5],
                                                  'n_estimators': [50, 100, 200, 300],
                                                        },
                                            cv=3,
                                            refit=True,
                                           n_jobs=-1))
                     ])

# 모델 훈련
reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('standard',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
  

In [43]:
# 훈련한 모델 파라미터
reg.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('standard',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEn

## 4. 데이터 예측

In [68]:
mod = submission.copy()

In [69]:
mod['Survived'] = reg.predict(test)

In [70]:
mod.to_csv('BART_titanic.csv', index=False)

## 무작위 내 데이터 넣어보기

In [87]:
lee = np.array([3, 'Lee', 'male', 24, 1, 2, 3019301, 10, np.nan, 'S'])

In [109]:
test.shape, np.shape(lee)
lee = lee.reshape(-1,10)

In [110]:
test = np.concatenate((test, lee), axis=0)

In [112]:
test = pd.DataFrame(test)

In [113]:
test.tail()

,0,1,2,3,4,5,6,7,8,9
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.25,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.05,NaN,S
417,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C
418,3,Lee,male,24,1,2,3019301,10,nan,S


In [115]:
train.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [116]:
test.columns = train.columns[1:]

In [117]:
predict = reg.predict(test)

In [121]:
predict[-1]

0